<center><h1> Optimizacion </h1></center>
<center><h2>Gradiente descendente</h2></center>

En este laboratoria se implementara el gradiente descendiente:

1. El metodo Steepest Decent con las condiciones de Armijo and Wolfe<br>
<br>
2. Metodod de Newton

Usaremos el paquete [autograd](https://github.com/HIPS/autograd) para calcular el gradiente y la Hessiana de nuestra funcion objetivo. <i>autograd</i> usa diferenciacion automatica. Basado en el lab del profesor 

In [ ]:
import numpy as np

#Use np.linalg.solve to solve A^-1*b

# to calculate the gradient and Hessian of the objective function
from autograd import grad, hessian

In [ ]:
def objective_function(x): 
    f = 0.5 * (x[0]**2 + 20 * x[1]**2)
    return f

In [ ]:
def rosenbrock(x):
    return ((x[0]-1)**2 + 100*(x[1]-x[0]**2)**2)

In [ ]:
def polynomial(x): 
    return - 1.0 / 6 * x[0]**6 + 1 / 4 *x[0]**4 + 2 * x[0]**2

In [ ]:
def gradient_function(func, x): 
    return grad(func)(x)

In [ ]:
def hessian_function(func,x):
    return hessian(func)(x)

In [ ]:
# input: (objective function, initial guess, step rule, c1, c2, initial alpha, optimality tolerance)
def steepest_descent(function, x0, stepRule, c1, c2, alpha0, tol=0.01):     
    
    xCur = x0.copy()
    fcur = function(xCur)
    gCur = gradient_function(function,xCur)
    
    # norm of the gradient at initial point for optimality condition
    nmg0 = np.linalg.norm(gradient_function(function, x0))
    
    # count number of steps taken by method
    it = 0
    
    # accumulate number of iterations needed by linesearch algorithm in every step
    ls_iter = 0
    
    # store iterates for plotting
    xIter=[]
    xIter.append(x0)
    
    while (np.linalg.norm(gCur)>tol*min(1,nmg0)): 

        it=it+1
        
        # calculate step-length
        if (stepRule=='armijo'):
            alpha, ls_ = armijo(function,xCur,-gCur, c1, alpha0)
        elif (stepRule=='wolfe'):
            alpha, ls_ = wolfe(function,xCur,-gCur, c1, c2, alpha0)
        else:
            alpha = 0.01
            ls_ = 1
        ls_iter = ls_iter + ls_
        
        # update current point
        #x(k+1) = x(k) + alpha*direction
        xCur = xCur + alpha*(-gCur)
        #print(xCur, alpha, direction)
        # add your code here
        fcur = function(xCur)
        gCur = gradient_function(function,xCur)

        xIter.append(xCur)

    return xCur, fcur, it, xIter, ls_iter

In [ ]:
# input: (objective function, initial guess, optimality tolerance)
def newton_descent(function, x0, stepRule='no', tol=0.01): 

    xCur = x0
    fcur = function(xCur)
    gCur = gradient_function(function,xCur)
    hCur = hessian_function(function,xCur)    
    
    # norm of the gradient at initial point for optimality condition
    nmg0 = np.linalg.norm(gradient_function(function, x0))    
    
    # count number of steps taken by method    
    it = 0
    
    # store iterates for plotting    
    xIter=[]
    xIter.append(x0)
    
    while(np.linalg.norm(gCur)>tol*min(1,nmg0)):
        
        it=it+1
        
        # calculate descent direction
        direction = # add your code here
        
        # calculate step-length or use natural length of 1
        if (stepRule ==  # add your code here
            alpha = # add your code here
        
        # update current point

        # add your code here
        
        xIter.append(xCur)

    return xCur, fcur, it, xIter

SyntaxError: ignored

In [ ]:
def armijo(function, xcur, searchdirection, c1, alpha0): 
    #f(x+alpha p) <= f(x) + c1 alpha grad(f).p
    
    alpha = alpha0
    xnew = xcur + alpha * searchdirection
    fcur = function(xcur) #phi(0)
    fnew = function(xnew) #phi(alpha)
    gradientCur = gradient_function(function,xcur) #phi'(0) = gradientCur.T@searchdirection
    numiter = 0
    # check for Armijo condition

    while (fnew > fcur + alpha*c1*gradientCur.T@searchdirection):
        numiter += 1
        alpha = alpha/2.0
        xnew = xcur + alpha * searchdirection
        fnew = function(xnew)

    return alpha, numiter

In [ ]:
def wolfe(function, xcur, searchdirection, c1, c2, alpha0): 
    #f(x+alpha p) \leq f(x) + c1 alpha grad(f)Tp
    #grad(f(x+alpha p))Tp \geq c2 grad(f)Tp
    
    alpha = alpha0
    xnew = xcur + alpha * searchdirection
    fcur = function(xcur)
    fnew = function(xnew)
    gradientCur = gradient_function(function,xcur)
    gradientNew = gradient_function(function,xnew)
    numiter = 0
    
    lb = 0
    ub = np.inf 
    
    # check for Wolfe conditions
    # adapted from https://sites.math.washington.edu/~burke/crs/408/notes/nlp/line.pdf (pg. 8)
    while True: 
        numiter += 1
        if ((fnew) > (fcur + c1 * alpha * np.dot(gradientCur,searchdirection))):
            ub = alpha
            alpha = 0.5 * (lb + ub)
        elif (np.dot(gradientNew,searchdirection) < c2 * np.dot(gradientCur,searchdirection)):
            lb = alpha
            if np.isinf(ub):
                alpha = 2 * lb
            else:
                alpha = 0.5 * (lb + ub)
        else:
            break
        xnew = xcur + alpha * searchdirection
        fnew = function(xnew)
        gradientNew = gradient_function(function,xnew)

    return alpha, numiter

## Solving quadratic unconstrained problem
###  Steepest descent

In [ ]:
x0 = np.array([0.9, 0.1])
c1 = 0.5 
c2 = 0.6 
alpha0 = 0.125 
tol = 1e-5

print ("{:<40} {:^20} {:^20} {:^15} {:^15}".format('Method','xopt','fopt','iter','ls_iter'))

# using steepest descent with Armijo condition for linesearch
xopt, fopt, it, xIter, ls_iter = steepest_descent(objective_function, x0, 'armijo', c1, c2, alpha0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Armijo with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")

#xopt, fopt, it, xIter, ls_iter = steepest_descent(objective_function, x0, 'wolfe', c1, c2, alpha0)
#print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Wolfe with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")

#xopt, fopt, it, xIter, ls_iter = steepest_descent(objective_function, x0, 'armijo', c1, c2, alpha0, tol)
#print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Armijo with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")


#xopt, fopt, it, xIter, ls_iter = steepest_descent(objective_function, x0, 'wolfe', c1, c2, alpha0, tol)
#print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Wolfe with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it,ls_iter))

Method                                           xopt                 fopt              iter           ls_iter    
Armijo with default tol = 0.01           [0.0069, -0.0001]          2.3644e-05           44              15             








### Newton's method

In [ ]:
print ("{:<40} {:^20} {:^20} {:^15}".format('Method','xopt','fopt','iter'))

xopt, fopt, it, xIter = newton_descent(objective_function, x0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it))
print("\n")

xopt, fopt, it, xIter = newton_descent(objective_function, x0, tol=0.0001)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it))

## Solving Rosenbrock's function
Try also different starting points and play around with the tuning factors.

### Steepest descent

In [ ]:
x0 = np.array([0.9, 0.1])
c1 = 0.5 
c2 = 0.6 
alpha0 = 0.125 
tol = 0.0001

print ("{:<40} {:^20} {:^20} {:^15} {:^15}".format('Method','xopt','fopt','iter','ls_iter'))

# using steepest descent with Armijo condition for linesearch
xopt, fopt, it, xIter, ls_iter = steepest_descent(rosenbrock, x0, 'armijo', c1, c2, alpha0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Armijo with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")

#xopt, fopt, it, xIter, ls_iter = steepest_descent(rosenbrock, x0, 'wolfe', c1, c2, alpha0)
#print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Wolfe with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")

#xopt, fopt, it, xIter, ls_iter = steepest_descent(rosenbrock, x0, 'armijo', c1, c2, alpha0, tol)
#print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Armijo with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it,ls_iter))

print("\n")


#xopt, fopt, it, xIter, ls_iter = steepest_descent(rosenbrock, x0, 'wolfe', c1, c2, alpha0, tol)
#print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d} {:<15d}".format('Wolfe with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it,ls_iter))

Method                                           xopt                 fopt              iter           ls_iter    
Armijo with default tol = 0.01           [0.9892, 0.9784]          1.1708e-04           964             5156           








### Newton's method

In [ ]:
print ("{:<40} {:^20} {:^20} {:^15}".format('Method','xopt','fopt','iter'))

xopt, fopt, it, xIter = newton_descent(rosenbrock, x0)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with default tol = 0.01',xopt[0],xopt[1],' ',fopt,it))
print("\n")

xopt, fopt, it, xIter = newton_descent(rosenbrock, x0, tol=0.0001)
print ("{:<40} [{:^6.4f}, {:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with tol = 0.0001',xopt[0],xopt[1],' ',fopt,it))

## Simple polynomial function
Here, a simple polynomial function is considered. 

In [ ]:
x0 = np.array([1.0])

xopt, fopt, it, xIter, ls_iter = steepest_descent(polynomial, x0, 'armijo', c1, c2, alpha0)
print ("{:<40} [{:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Steepest descent with default tol = 0.01',xopt[0],' ',fopt,it))

xopt, fopt, it, xIter = newton_descent(polynomial, x0, stepRule='armijo')
print ("{:<40} [{:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with Armijo default tol = 0.01',xopt[0],' ',fopt,it))

xopt, fopt, it, xIter = newton_descent(polynomial, x0)
print ("{:<40} [{:^6.4f}] {:<8} {:<20.4e} {:<15d}".format('Newton with default tol = 0.01',xopt[0],' ',fopt,it))

